# Case study: mailing list manager
The manager keeps track of email addresses categorized into named groups. When it's time to send a message, we can pick a group and send the message to all email addresses assigned to that group

In [2]:
import smtplib
from email.mime.text import MIMEText

In [3]:
def send_email(subject,
               message,
               from_addr,
               *to_addrs,
               host="localhost",
               port=1025,
               **headers):
    email = MIMEText(message)
    email["Subject"] = subject
    email["From"]=from_addr
    for header, value in headers.items():
        email[header] = value
    
    sender = smtplib.SMTP(host, port)
    for addr in to_addrs:
        del email["To"]
        email["To"]=addr
        sender.sendmail(from_addr, addr, email.as_string())
    sender.quit()

Since the values in our dictionary will always be collections of unique email addresses, we can store them in a `set` container. We can use `defaultdict` to ensure that there is always a `set` container available for each key, demonstrated as follows below:

In [9]:
from collections import defaultdict

class MailingList:
    """Manage groups of e-mail addresses for sending e-mails."""
    def __init__(self):
        self.email_map = defaultdict(set)
        
    def add_to_group(self, email, group):
        self.email_map[email].add(group)
    
    def emails_in_groups(self, *groups): 
        groups = set(groups)
        emails = set() 
        for e, g in self.email_map.items(): 
            if g & groups: 
                emails.add(e)
        return emails
    
    def send_mailing(self,
                     subject,
                     message,
                     from_addr,
                     *groups,
                     headers=None):
        emails = self.emails_in_groups(*groups)
        send_email(subject, message, from_addr, *emails, header=headers)
    